In [1]:
import dask.bag as db
import re

## Pride and prejudice from  
`http://www.gutenberg.org`  

Book Link : https://www.gutenberg.org/files/1342/1342-0.txt  
Jane Austin's Project Gutenberg page : https://www.gutenberg.org/ebooks/author/68

In [2]:
%pip install requests


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# FIXME 
# install certifi?
# URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)>
# https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org
# The workaround works in venv but not here.

# import dask 
# dask.config.set({"distributed.scheduler.http.request_kwargs": {"verify": False}}) # tried. didn't work

# pem_path = "/etc/ssl/cert.pem"
# dask.config.set({"distributed.scheduler.http.request_kwargs": {"verify": "/etc/ssl/cert.pem"}}) 

# WORKAROUND 
# see get_data script

# does not work. same error
# import requests
# response = requests.get('https://www.gutenberg.org/files/1342/1342-0.txt') 


data_dir = "/Users/alexiuk/projects/data/gutenberg/"
with open(data_dir + "pride_and_prejudice.txt") as f:
    text = f.read()

book_bag = db.from_sequence(text)

# orig
# book_bag = db.from_url('https://www.gutenberg.org/files/1342/1342-0.txt')

In [6]:
book_bag.take(5)

('ï', '»', '¿', 'T', 'h')

In [7]:
strip_spaces = book_bag.map(lambda x:x.strip())

In [8]:
strip_spaces.take(10)

('ï', '»', '¿', 'T', 'h', 'e', '', 'P', 'r', 'o')

In [9]:
def decode_to_ascii(x):
    return x.decode("ascii","ignore") 

In [10]:
ascii_text = strip_spaces.map(decode_to_ascii)

In [11]:
ascii_text.take(10)

AttributeError: 'str' object has no attribute 'decode'

In [12]:
def remove_punctuation(x):
    return re.sub(r'[^\w\s]','',x)

In [13]:
remove_punctuation = ascii_text.map(remove_punctuation)

In [14]:
remove_punctuation.take(10)

AttributeError: 'str' object has no attribute 'decode'

In [15]:
lower_text = remove_punctuation.map(str.lower)

In [16]:
lower_text.take(10)

AttributeError: 'str' object has no attribute 'decode'

In [17]:
split_word_list = lower_text.map(lambda x: x.split(' '))

In [18]:
split_word_list.take(10)

AttributeError: 'str' object has no attribute 'decode'

In [19]:
def remove_empty_words(word_list):
    return list(filter(lambda a: a != '', word_list))

non_empty_words = split_word_list.filter(remove_empty_words)

In [20]:
non_empty_words.take(10)

AttributeError: 'str' object has no attribute 'decode'

In [ ]:
all_words = non_empty_words.flatten()

In [ ]:
type(all_words)

dask.bag.core.Bag

In [ ]:
all_words.take(30)

('the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'pride',
 'and',
 'prejudice',
 'by',
 'jane',
 'austen',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever',
 '')

In [ ]:
change_to_key_value = all_words.map(lambda x: (x, 1))

In [ ]:
change_to_key_value.take(4)

(('the', 1), ('project', 1), ('gutenberg', 1), ('ebook', 1))

## Using `groupby`

In [ ]:
grouped_words = all_words.groupby(lambda x:x)

In [ ]:
grouped_words.take(1)

(('the',
  ['the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
   'the',
 

In [ ]:
word_count = grouped_words.map(lambda x: (x[0], len(x[1])))

In [ ]:
word_count.take(10)

(('the', 4496),
 ('project', 88),
 ('gutenberg', 30),
 ('ebook', 11),
 ('of', 3716),
 ('pride', 50),
 ('and', 3602),
 ('prejudice', 11),
 ('by', 657),
 ('jane', 263))

## Using `foldby`

In [ ]:
change_to_key_value.take(10)

(('the', 1),
 ('project', 1),
 ('gutenberg', 1),
 ('ebook', 1),
 ('of', 1),
 ('pride', 1),
 ('and', 1),
 ('prejudice', 1),
 ('by', 1),
 ('jane', 1))

In [ ]:
# Take a running count of a word
# In this case, the default value of 
# count needs to be provided
def add_bin_op(count, x):
    return count + x[1]

# Take the output from multiple bin_op(s)
# and add them to get the total count of
# a word
def add_combine_op(x, y):
    return x + y

word_count = change_to_key_value.foldby(lambda x: x[0],
                                       add_bin_op, 0,
                                       add_combine_op)

In [ ]:
word_count.take(10)

(('the', 4496),
 ('project', 88),
 ('gutenberg', 30),
 ('ebook', 11),
 ('of', 3716),
 ('pride', 50),
 ('and', 3602),
 ('prejudice', 11),
 ('by', 657),
 ('jane', 263))

## Using `frequencies`

In [ ]:
much_easier = all_words.frequencies()

In [ ]:
much_easier.take(10)

(('the', 4496),
 ('project', 88),
 ('gutenberg', 30),
 ('ebook', 11),
 ('of', 3716),
 ('pride', 50),
 ('and', 3602),
 ('prejudice', 11),
 ('by', 657),
 ('jane', 263))

We see that there are stopwords with most frequencies. Lets remove them.

In [ ]:
from spacy.lang.en import STOP_WORDS

In [ ]:
without_stopwords = all_words.filter(lambda x: x not in STOP_WORDS)

In [ ]:
new_freq = without_stopwords.frequencies()

In [ ]:
new_freq.take(20)

(('project', 88),
 ('gutenberg', 30),
 ('ebook', 11),
 ('pride', 50),
 ('prejudice', 11),
 ('jane', 263),
 ('austen', 4),
 ('use', 26),
 ('cost', 7),
 ('restrictions', 2),
 ('whatsoever', 2),
 ('', 98),
 ('copy', 12),
 ('away', 119),
 ('reuse', 2),
 ('terms', 43),
 ('license', 15),
 ('included', 5),
 ('online', 4),
 ('wwwgutenbergorg', 3))

In [ ]:
new_freq.topk(10)

dask.bag<topk-ag..., npartitions=1>

In [ ]:
new_freq.topk(10, key=lambda x: x[1]).compute()

[('mr', 782),
 ('elizabeth', 594),
 ('said', 401),
 ('darcy', 370),
 ('mrs', 343),
 ('bennet', 293),
 ('miss', 283),
 ('jane', 263),
 ('bingley', 257),
 ('know', 237)]